In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

from sklearn.metrics import mean_absolute_percentage_error as mean_ape
from sklearn.metrics import root_mean_squared_error as root_mse

In [2]:
model_data = pd.read_parquet("data/model_data.parquet")
model_data = model_data[model_data['alignement_politique'] != 'autre']

model_data['pres_votes_share'] = model_data['pres_dummy'] * model_data['pres_votes_share']

In [3]:
nuances_order = [
    "extremedroite_droiteradicale",
    "centredroite_droite",
    "centre",
    "centregauche_gauche",
    "extremegauche_gaucheradicale"
]

Approche économétrique classique : on modélise des comportements réels.
On s'intéresse principalement aux coefficients et à leur significativité.

$R^2$ n'est pas une mesure appropriée pour comparer des modèles ajustés sur des échantillons différents, et en l'absence de constante.

Il faut mesurer directement la distance entre les valeurs réelles et prédites.
De préférence en normalisant, car les valeurs prédites sont assez différentes entre les partis : un écart moyen de 5 pp. entre les valeurs réelles et prédites est mineur lorsque les valeurs prédites gravitent autour de 50 %, majeur lorsqu'elles gravitent autour de 5 %.
- *Mean absolute percentage error (MAPE)* : $\frac{1}{n} \sum_{i=1}^{n} \left| \frac{\hat{y}_i - y_i}{y_i} \right|$

    S'interprète directement : une MAPE de 1 signifie que les valeurs prédites s'écartent en moyenne de 100 % des valeurs réelles.
- *Root mean squared percentage error (RMSPE)*: $\sqrt{\frac{1}{n} \sum_{i=1}^{n} \left(\frac{\hat{y}_i - y_i}{y_i}\right)^2}$

    Ne s'interprète pas directement. Permet seulement de comparer les modèles. Plus sensible aux outliers que la MAPE.

Elimination des constantes, qui rendent les coefficients beaucoup plus difficiles à interpréter sans améliorer sensiblement les prédictions (on se retrouve par exemple avec des constantes très élevées, et des coefficients négatifs pour les résultats électoraux).

## 1. Modèle de base
De la forme $Y_{i} = \beta T_{i}$ avec :
- $Y_{i}$ la proportion des articles intégrant des citations de la nuance politique $i$ (avec $\sum_{i=1}^{n} Y_{i} = 1$)
- $\beta$ la pondération des résultats électoraux de chaque nuance politique
- $T_{i}$ la proportion des voix obtenue par les candidats de la nuance politique $i$ au premier tour des précédentes élections législatives

In [4]:
X = model_data[["leg_votes_share"]]
y = model_data["quotes_share"]
model = sm.OLS(y, X).fit(cov_type='HC3')
y_pred = model.predict(X)
mape = mean_ape(y, y_pred)
rmspe = root_mse(y, y_pred) / y.mean()

params = model.params.rename("coef").to_frame()
pvalues = model.pvalues.rename("pval").to_frame()
print(pd.merge(params, pvalues, left_index=True, right_index=True))
print("")
print(f"MAPE: {100*mape:.2f}%")
print(f"RMSPE: {rmspe:.5f}")

                     coef  pval
leg_votes_share  1.771711   0.0

MAPE: 107.43%
RMSPE: 0.50714


## 2. Modèle de base + gouvernement
De la forme $Y_{i} = \alpha + \beta T_{i} + \gamma G_{i}$ avec :
- $\gamma$ la proportion des articles intégrant des citations dont bénéficie la nuance politique du gouvernement
- $G_{i}$ une indicatrice valant 1 si le Premier ministre appartient à la nuance politique $i$

In [7]:
X = model_data[["leg_votes_share", "pres_votes_share", "government"]]
y = model_data["quotes_share"]
model = sm.OLS(y, X).fit(cov_type='HC3')
y_pred = model.predict(X)
mape = mean_ape(y, y_pred)
rmspe = root_mse(y, y_pred) / y.mean()

params = model.params.rename("coef").to_frame()
pvalues = model.pvalues.rename("pval").to_frame()
print(pd.merge(params, pvalues, left_index=True, right_index=True))
print("")
print(f"MAPE: {100*mape:.2f}%")
print(f"RMSPE: {rmspe:.5f}")

                      coef      pval
leg_votes_share   1.370668  0.000000
pres_votes_share  0.020140  0.523663
government        0.185165  0.000000

MAPE: 81.62%
RMSPE: 0.42806


## 3. Modèle de base + gouvernement + nuances politiques
Les modèles estimés sont de la forme $Y_{i} = \beta_{i}T_{i} + \gamma_{i} G_{i}$ avec :
- $\beta_{i}$ la pondération des résultats électoraux de la nuance politique $i$
- $\gamma_{i}$ la couverture médiatique supplémentaire dont bénéficie la nuance politique $i$ lorsqu'elle est au gouvernement

In [23]:
models = {}

for alignement in model_data["alignement_politique"].unique():
    subset = model_data[model_data["alignement_politique"] == alignement]

    if len(subset) >= 3:
        X = subset[["leg_votes_share", "government"]]
        y = subset["quotes_share"]
        model = sm.OLS(y, X).fit(cov_type='HC3')      
        y_pred = model.predict(X)
        mape = mean_ape(y, y_pred)
        rmspe = root_mse(y, y_pred) / y.mean()
        
        models[alignement] = {
            "params": model.params.rename("params").to_frame(),
            "pvalues": model.pvalues.rename("pval").to_frame(),
            "mape": mape,
            "rmspe": rmspe}

    else:
        models[alignement] = {
            "params": None,
            "pvalues": None,
            "mape": None,
            "rmspe": None}

In [24]:
summary = []

for alignement, model_info in models.items():
    summary.append({
        "alignement": alignement,
        "mape": model_info["mape"],
        "rmspe": model_info["rmspe"]
    })

results = pd.DataFrame(summary)
results["alignement"] = pd.Categorical(results["alignement"], categories=nuances_order, ordered=True)
results = results.sort_values("alignement").reset_index(drop=True)

results.style.hide(axis="index") 

alignement,mape,rmspe
extremedroite_droiteradicale,0.746741,0.911634
centredroite_droite,0.228974,0.238748
centre,0.950210,0.544169
centregauche_gauche,0.298521,0.350523
extremegauche_gaucheradicale,0.792376,1.022237


In [25]:
results_list = []

for alignement, model_info in models.items():
    df = model_info["params"].join(model_info["pvalues"])
    df["alignement"] = alignement
    df["variable"] = df.index
    results_list.append(df.reset_index(drop=True))

results = pd.concat(results_list, ignore_index=True)
results = results.dropna(subset=["pval"])
results["alignement"] = pd.Categorical(results["alignement"], categories=nuances_order, ordered=True)
results = results.sort_values(["variable", "alignement"])
results = results.set_index(["variable", "alignement"])
results.index.names = ["Variable", "Alignement"]

results.style.format("{:.4f}")

## 4. Modèle de base + gouvernement + nuances politiques + score aux présidentielles
Les modèles estimés sont de la forme $Y_{i} = \beta_{i}T_{i} + \gamma_{i} G_{i} + \theta_{i} P_{i}$ avec :
- $\beta_{i}$ la pondération des résultats électoraux de la nuance politique $i$
- $\gamma_{i}$ la couverture médiatique supplémentaire dont bénéficie la nuance politique $i$ lorsqu'elle est au gouvernement
- $\theta_{i}$ la pondération des résultats aux élections présidentielles de la nuance politique $i$
- $P_{i}$ les résultats de la nuance politique $i$ à l'élection présidentielle suivante (pour le mois de l'élection et les trois précédents)

In [26]:
models = {}

for alignement in model_data["alignement_politique"].unique():
    subset = model_data[model_data["alignement_politique"] == alignement]

    if len(subset) >= 3:
        X = subset[["leg_votes_share", "pres_votes_share", "government"]]
        y = subset["quotes_share"]
        model = sm.OLS(y, X).fit(cov_type='HC3')      
        y_pred = model.predict(X)
        mape = mean_ape(y, y_pred)
        rmspe = root_mse(y, y_pred) / y.mean()
        
        models[alignement] = {
            "params": model.params.rename("params").to_frame(),
            "pvalues": model.pvalues.rename("pval").to_frame(),
            "mape": mape,
            "rmspe": rmspe}

    else:
        models[alignement] = {
            "params": None,
            "pvalues": None,
            "mape": None,
            "rmspe": None}

In [27]:
summary = []

for alignement, model_info in models.items():
    summary.append({
        "alignement": alignement,
        "mape": model_info["mape"],
        "rmspe": model_info["rmspe"]
    })

results = pd.DataFrame(summary)
results["alignement"] = pd.Categorical(results["alignement"], categories=nuances_order, ordered=True)
results = results.sort_values("alignement").reset_index(drop=True)

results.style.hide(axis="index") 

alignement,mape,rmspe
extremedroite_droiteradicale,0.698947,0.873674
centredroite_droite,0.228273,0.237903
centre,0.956768,0.535826
centregauche_gauche,0.297281,0.350319
extremegauche_gaucheradicale,0.782780,1.004210


In [28]:
results_list = []

for alignement, model_info in models.items():
    df = model_info["params"].join(model_info["pvalues"])
    df["alignement"] = alignement
    df["variable"] = df.index
    results_list.append(df.reset_index(drop=True))

results = pd.concat(results_list, ignore_index=True)
results = results.dropna(subset=["pval"])
results["alignement"] = pd.Categorical(results["alignement"], categories=nuances_order, ordered=True)
results = results.sort_values(["variable", "alignement"])
results = results.set_index(["variable", "alignement"])
results.index.names = ["Variable", "Alignement"]

results.style.format("{:.4f}")

## 5. Modèle de base + gouvernement + nuances politiques + score aux présidentielles + périodes
Il s'agit maintenant d'étudier plus directement l'hypothèse d'une légitimation de l'extrême droite par la presse écrite nationale. La méthode la plus simple est d'estimer les modèles précédents pour 2 périodes, afin d'observer l'évolution des coefficients. Cela s'écrit $Y_{it} = \beta_{it} T_{it} + \gamma_{it} G_{it}$ en désignant par $t$ les périodes).

On retient 2012 comme charnière, avec l'idée que...
- La période précédente est encore marquée par le jeu fonctionnement traditionnel du système politique, avec une domination persistante de la droite et de la gauche de gouvernement, malgré leurs recompositions.
- La période suivante est caractérisée par l'effondrement de ce système, avec l'essort du centre et des extrêmes.

In [30]:
cutoff = pd.Period('2012-06', freq='M')

models = {}

for period_label, period_filter in {
    'pre2012': model_data["month"] < cutoff,
    'post2012': model_data["month"] >= cutoff
}.items():
    
    models[period_label] = {}
    period_data = model_data[period_filter]
    
    for alignement in period_data["alignement_politique"].unique():
        models[period_label][alignement] = {}
        subset = period_data[period_data["alignement_politique"] == alignement]

        if len(subset) >= 3:
            X = subset[["leg_votes_share", "pres_votes_share", "government"]]
            y = subset["quotes_share"]            
            model = sm.OLS(y, X).fit(cov_type='HC3')
            y_pred = model.predict(X)
            mape = mean_ape(y, y_pred)
            rmspe = root_mse(y, y_pred) / y.mean()

            models[period_label][alignement] = {
                "params": model.params.rename("params").to_frame(),
                "pvalues": model.pvalues.rename("pval").to_frame(),
                "mape": mape,
                "rmspe": rmspe}

        else:
            models[period_label][alignement] = {
                "params": None,
                "pvalues": None,
                "mape": None,
                "rmspe": None}

In [31]:
summary_list = []

for period, alignement_dict in models.items():
    for alignement, model_info in alignement_dict.items():
        summary_list.append({
            "period": period,
            "alignement": alignement,
            "mape": model_info["mape"],
            "rmspe": model_info["rmspe"]
        })

summary = pd.DataFrame(summary_list)
summary["alignement"] = pd.Categorical(summary["alignement"], categories=nuances_order, ordered=True)
summary = summary.pivot_table(index="alignement",
                                      columns="period",
                                      values=["mape", "rmspe"],
                                      observed=False)
summary.columns = summary.columns.swaplevel(0, 1)
summary = summary.reindex(columns=["pre2012", "post2012"], level=0)
summary = summary.reset_index()
summary.columns.names = [None, None]
summary = summary.sort_values("alignement").reset_index(drop=True)

format_cols = summary.columns[1:]
summary.style.hide(axis="index").format({col: "{:.3f}" for col in format_cols})

In [32]:
results_list = []

for period, alignement_dict in models.items():
    for alignement, model_info in alignement_dict.items():
        df = model_info["params"].join(model_info["pvalues"])
        df["period"] = period
        df["alignement"] = alignement
        df["variable"] = df.index
        results_list.append(df.reset_index(drop=True))

results = pd.concat(results_list, ignore_index=True)
results = results.dropna(subset=["pval"])
results["alignement"] = pd.Categorical(results["alignement"], categories=nuances_order, ordered=True)
results = results.pivot_table(index=["alignement", "variable"],
                              columns="period",
                              values=["params", "pval"],
                              observed=False)
results.columns = results.columns.swaplevel(0, 1)
results = results.reindex(columns=["pre2012", "post2012"], level=0)
results = results.sort_index()
results.index.names = ["Alignement", "Variable"]

results.style.format({col: "{:.4f}" for col in results.columns})

## 6. Modèle de base + gouvernement + nuances politiques + périodes + journaux
On différencie finalement selon les journaux. En les indiçant par $j$, les modèles s'écrivent : $Y_{ijt} = \beta_{ijt}T_{it} + \gamma_{it} G_{it}$

In [37]:
cutoff = pd.Period('2012-06', freq='M')

models = {}

for period_label, period_filter in {
    'pre2012': model_data["month"] < cutoff,
    'post2012': model_data["month"] >= cutoff
}.items():
    
    models[period_label] = {}
    period_data = model_data[period_filter]
    
    for alignement in period_data["alignement_politique"].unique():
        models[period_label][alignement] = {}
        subset = period_data[period_data["alignement_politique"] == alignement]
        
        for journal in period_data["journal"].unique():
            sub_subset = subset[subset["journal"] == journal]

            if len(subset) >= 3:
                X = sub_subset[["leg_votes_share", "government"]]
                y = sub_subset["quotes_share"]            
                model = sm.OLS(y, X).fit(cov_type='HC3')
                y_pred = model.predict(X)
                mape = mean_ape(y, y_pred)
                rmspe = root_mse(y, y_pred) / y.mean()

                models[period_label][alignement][journal] = {
                    "params": model.params.rename("params").to_frame(),
                    "pvalues": model.pvalues.rename("pval").to_frame(),
                    "mape": mape,
                    "rmspe": rmspe}

            else:
                models[period_label][alignement][journal] = {
                    "params": None,
                    "pvalues": None,
                    "mape": None,
                    "rmspe": None}

In [38]:
summary = []

for period, alignement_dict in models.items():
    for alignement, journal_dict in alignement_dict.items():
        for journal, model_info in journal_dict.items():
            summary.append({
                "period": period,
                "alignement": alignement,
                "journal": journal,
                "mape": model_info["mape"],
                "rmspe": model_info["rmspe"]
            })

results = pd.DataFrame(summary)
results["alignement"] = pd.Categorical(results["alignement"], categories=nuances_order, ordered=True)
results = results.pivot_table(index=["alignement", "journal"],
                              columns="period",
                              values=["mape", "rmspe"],
                              observed=False)
results.columns = results.columns.swaplevel(0, 1)
results = results.reindex(columns=["pre2012", "post2012"], level=0)
results = results.sort_index()
results.index.names = ["Alignement", "Journal"]

format_cols = results.columns
results.style.format({col: "{:.3f}" for col in format_cols})

In [39]:
results_list = []

for period, alignement_dict in models.items():
    for alignement, journal_dict in alignement_dict.items():
        for journal, model_info in journal_dict.items():
            df = model_info["params"].join(model_info["pvalues"])
            df["period"] = period
            df["alignement"] = alignement
            df["journal"] = journal
            df["variable"] = df.index
            results_list.append(df.reset_index(drop=True))

results = pd.concat(results_list, ignore_index=True)
results = results.dropna(subset=["pval"])
results["alignement"] = pd.Categorical(results["alignement"], categories=nuances_order, ordered=True)
results = results.pivot_table(index=["variable", "alignement", "journal"],
                              columns="period",
                              values=["params", "pval"],
                              observed=False)

results.columns = results.columns.swaplevel(0, 1)
results = results.reindex(columns=["pre2012", "post2012"], level=0)
results = results.sort_index()
results.index.names = ["Variable", "Alignement", "Journal"]

results.style.format({col: "{:.4f}" for col in results.columns})